In [2]:
# wd = '/mnt/openfact/users/msawinski/factue-task2'
wd = '/Users/marcinsawinski/Documents/GitHub/factue-task2'
import sys, os
os.chdir(wd)

In [15]:
import pandas as pd
from pathlib import Path
import os
import json
from factue.utils.viz import disp
import numpy as np

# finetune


In [10]:
root = Path("data/cleaned/normalization")
# Example path template
files = root.rglob("**/*.parquet")

# Read all files and add file path as a column
df_list = []
for f in files:
    df_part = pd.read_parquet(f)
    df_part['source_file'] = f  # add the file path
    df_list.append(df_part)

# Combine into one DataFrame
df = pd.concat(df_list, ignore_index=True)
df['split'] = df.source_file.astype(str).str.split('/',expand=True)[4]
df

,text,reference,split,lang,init_part_no,source_file
0,"INZ\nPESQUISA IBOPE\nAprovação de\nBolsonaro vai a80%\nSe você não viralizar, a\ngrande mídia não fará isso\npor você.",Bolsonaro tem 80% de aprovação no Ibope,train-por,train-por,25,data/cleaned/normalization/train/train-por/train-por_part_0001.parquet
1,"Este é um cemitério perto de Paris, França, com\ncentenas de carros elétricos. Lembre-se de que estes\nsão apenas carros usados pela cidade de Paris e não\nveículos pessoais. Todos eles têm o mesm...",Baterias de carros elétricos drenam toxinas para o solo,train-por,train-por,28,data/cleaned/normalization/train/train-por/train-por_part_0001.parquet
2,😏 😏 😏 Notícias on line\nN\nNOTICIAS\nIMAGE\nPaolla Oliveira diz que a\nprostituição será a única forma de\nsobrevivência das atrizes da Globo\ncaso Bolsonaro seja reeleito,Paolla Oliveira disse que atrizes da Globo terão de se prostituir para sobreviver caso Bolsonaro seja reeleito,train-por,train-por,59,data/cleaned/normalization/train/train-por/train-por_part_0001.parquet
3,"Isso aqui é uma delegacia de Minneapolis, Minnesota ( EUA) pegando fogo devido ao assassinato de George Floyd, homem negro assassinado brutalmente e injustamente pela policial.\nIsso aqui é fogo n...",Delegacia de polícia em Minneapolis em chamas depois da morte de George Floyd,train-por,train-por,37,data/cleaned/normalization/train/train-por/train-por_part_0001.parquet
4,"Esta é a primeira imagem de ressonância magnética do mundo que mostra o laço de uma mãe e seu filho, graças à NH Neuro Training. A imagem é da neurocientista Rebecca Saxe a beijar o seu filho de d...",Ressonância magnética de uma mãe com seu filho mostra reação química nos dois cérebros por um beijo,train-por,train-por,53,data/cleaned/normalization/train/train-por/train-por_part_0001.parquet
...,...,...,...,...,...,...
7500,"Had gone to get India closed. In Papalaj, Rajasthan, the public thrashed him badly and blackened his face.\n","Had gone to get India closed. In Papalaj, Rajasthan, the public thrashed him badly and blackened his face.\n",dev-eng,dev-eng,53,data/cleaned/normalization/dev/dev-eng/dev-eng_part_0037.parquet
7501,A Muslim youth molested a Hindu woman in the market.\n \n #AdaniBackOnTrack #love #StopDrinkingAlcohol,A Muslim youth molested a Hindu woman in the market,dev-eng,dev-eng,40,data/cleaned/normalization/dev/dev-eng/dev-eng_part_0037.parquet
7502,"🔥Hundreds of Muslims shouting to stop Poojas at Kali Matha Mandir in Kolkata, once poojas conducted by Sri Ramakrishna Paramahamsa. They (so-called peace lovers) want to close the Mandir. What is ...",Muslims protesting against temple in Kolakata,dev-eng,dev-eng,4,data/cleaned/normalization/dev/dev-eng/dev-eng_part_0037.parquet
7503,Meet Lebo Moatshe from South Africa who successfully removed HIV from a sperm cell in a lab which is a huge medical breakthrough. She has done 20 semen decontamination procedures (removing HIV fro...,A South African scientist made a “huge medical breakthrough” by successfully removing HIV from a sperm cell in a lab,dev-eng,dev-eng,13,data/cleaned/normalization/dev/dev-eng/dev-eng_part_0037.parquet


In [12]:
from sklearn.model_selection import train_test_split
from factue.methods.llm_calls import load_template_list, load_template_parts

# Split 80% fine-tune, 20% eval
df_finetune, df_eval = train_test_split(df, test_size=0.2, random_state=42)

In [21]:
SYSTEM_PART_NAME = "system"
USER_PART_NAME = "user"

def build_messages(post,reference,job="normalization", step="extract", prompt_name="default", prompt_version="v001"):
    prompt_layout, prompt_content = load_template_parts(
        job=job, step="extract", prompt_name=prompt_name, prompt_version=prompt_version
    )

    json_line = {
            "messages": [
                {"role": "system", "content": prompt_layout[SYSTEM_PART_NAME].format(instructions=prompt_content['instructions'])},
                {"role": "user", "content": prompt_layout[USER_PART_NAME].format(post = post)},
                {"role": "assistant", "content": reference}
            ]
        }
    return json_line
    # return(json.dumps(json_line, ensure_ascii=False) + "\n")


In [22]:
# Write DataFrame to JSONL
ft_df = df_finetune
with open("data/finetuning/normalize_train.jsonl", "w", encoding="utf-8") as file:
    for _, row in ft_df.iterrows():
        json_line = build_messages(post = row['text'],reference = row['reference'])
        file.write(json.dumps(json_line, ensure_ascii=False) + "\n")

In [23]:
# Write DataFrame to JSONL
ft_df = df_eval
with open("data/finetuning/normalize_eval.jsonl", "w", encoding="utf-8") as file:
    for _, row in ft_df.iterrows():
        json_line = build_messages(post = row['text'],reference = row['reference'])
        file.write(json.dumps(json_line, ensure_ascii=False) + "\n")